In [197]:
import pandas as pd
import json
import re
import numpy as np
import ast
from pathlib import Path  

In [198]:
main_path = "/home/rvissche/Nextcloud/What-If/what-if-data-donation/what-if-data-donation/structure_donations/Processed_structure_donations/"

In [199]:
# Define the data types
data_types = ['string', 'array', 'number', 'boolean', 'object', 'str', 'int', 'float', 'bool', 'dict', 'list']
columns = ['col_path_1','col_path_2','col_path_3','col_path_4', 'col_path_5']


def process_row_path(row, columns, data_types):
    row['data_type'] = ''
    for column in columns:
        if row[column] in data_types:
            row['data_type'] = row[column]
            row[column] = np.nan
        else:
            row[column]
    return row


In [200]:
def file_paths(df):
    
    # Create a column with the JSON name
    #df['json_name'] = df['variable'].str.rsplit('/', n=1).str[-1]
    # As the JSON name is stored in the json_name column, fill other parts of the path with Na if the name of the JSON is present
    #mark = ".json"

    # Unlist the value column (where the JSON info is stored)
    for index, row in df.iterrows():
        if isinstance(row['value'], list):
            df.at[index, 'value'] = row['value'][0]
    
    return df

In [201]:
def string_to_dict(s):
    if ',' not in s and ':' not in s:
        return s
    result = {}
    items = s.split(',')
    for item in items:
        if ':' not in item:
            continue  
        key, value = item.split(':', 1)  # use maxsplit=1 to avoid unpacking issues
        try:
            key = eval(key.strip())
            value = eval(value.strip())
        except Exception as e:
            continue
        result[key] = value
    return result

    

In [202]:
def row_column_paths(df):
    # Initialize the new columns
    df['row_path'] = df['variable'] 
    df['col_path_1'] = ''  
    df['col_path_2'] = ''  
    df['col_path_3'] = ''
    df['col_path_4'] = ''
    df['col_path_5'] = '' 

    
    """
    # Extract the level 1 keys 
    for index, row in df.iterrows():
        df.at[index, 'row_path'] = list(row['value'].keys())

    df['row_path'] = df['row_path'].astype('str').apply(eval).str[0]
   
    # Take the level 1 keys stored in a list and store them in individual rows
    df = df.explode('row_path')

    """


    def column_paths(df, row_var, get_var, unlist_var, explode_var):

        # Convert to dict if it's a string
        df[row_var] = df[row_var].apply(
            lambda x: string_to_dict(x) if isinstance(x, str) else x)
        
        
    
        df[get_var] = df.apply(
        lambda row: list(row[row_var].keys()) if isinstance(row[row_var], dict) else row[row_var], axis=1)


        #display(df)

       
        df = df.explode(get_var)

        
        # Extract nested values using keys
        df[unlist_var] = df.apply(lambda row: [row[row_var].get(row[get_var], None)] if isinstance (row[row_var], dict) else None, axis=1)

    

        # Unlist and explode
        df[unlist_var] = df[unlist_var].apply(lambda x: x[0] if isinstance(x, list) else x)
        df[unlist_var] = df[unlist_var].apply(lambda x: x[0] if isinstance(x, list) else x)

        """
        
        df[unlist_var] = df[unlist_var].apply(lambda x: x[0] if isinstance(x, list) else x)

        #display(df[unlist_var])
        
        df[explode_var] = df[unlist_var]
        df = df.explode(explode_var)
        """

    
        return df

    row_var = ['value', 'col_path_1_values', 'col_path_2_values', 'col_path_3_values', 'col_path_4_values']
    get_var = ['col_path_1', 'col_path_2', 'col_path_3', 'col_path_4', 'col_path_5']
    unlist_var = ['col_path_1_values', 'col_path_2_values', 'col_path_3_values', 'col_path_4_values', 'col_path_5_values']
    explode_var = ['col_path_1', 'col_path_2', 'col_path_3', 'col_path_4', 'col_path_5']


    for r, g, u, e in zip(row_var, get_var, unlist_var, explode_var):
        df = column_paths(df, r, g, u, e)
        
    return df


In [203]:
def clean_and_store(df, file_name):
 
    # Reorder the columns in the df
    df = df.loc[:, ['variable', 'value', 'row_path', 'col_path_1','col_path_2','col_path_3', 'col_path_4', 'col_path_5','data_type', 'col_path_1_values','col_path_2_values','col_path_3_values', 'col_path_4_values']]
    
    df.reset_index(drop=True, inplace=True)
    df = df.fillna('Missing')
    #df = df.astype(str)
    


    df.to_csv(f"{main_path}TikTok/Output/Output_" + file_name + '.csv', index=False)

    return df

In [204]:
def structure_donations(data):

    data = Path(data)  
    #file_name = data.name  # Extracts "data.json"
    file_name = Path(data).stem 

    # Load JSON file
    with open(data, 'r') as f:
        data = json.load(f)
    
    # Strip top-level key
    data = list(data.values())[0]


    # Flatten JSON (handling nested structures)
    df = pd.json_normalize(data, max_level=0)



    # Delete user specific informations
    #df.columns = df.columns.str.replace(r'^[^/]+/', '', regex=True)

    # Extract column names
    cols = df.columns.tolist()


    # From wide to long df
    df = pd.melt(df, value_vars= cols)


    df = file_paths(df)
    
    df = row_column_paths(df)
    
    df = df.apply(lambda row: process_row_path(row, columns, data_types), axis=1)

    df = clean_and_store(df, file_name)
    
    display(df)
    
    return df


In [205]:
input_directory = Path(f'{main_path}TikTok/Input_test')  
print(input_directory)

/home/rvissche/Nextcloud/What-If/what-if-data-donation/what-if-data-donation/structure_donations/Processed_structure_donations/TikTok/Input_test


In [206]:

for file in input_directory.iterdir():  
    if file.is_file():  
        #print(file)
        structure_donations(file)
        

,variable,value,row_path,col_path_1,col_path_2,col_path_3,col_path_4,col_path_5,data_type,col_path_1_values,col_path_2_values,col_path_3_values,col_path_4_values
0,Activity Summary,"{'ActivitySummaryMap': {'note': 'str', 'videos...",Activity Summary,ActivitySummaryMap,note,Missing,Missing,Missing,str,"{'note': 'str', 'videosCommentedOnSinceAccount...",str,Missing,Missing
1,Activity Summary,"{'ActivitySummaryMap': {'note': 'str', 'videos...",Activity Summary,ActivitySummaryMap,videosCommentedOnSinceAccountRegistration,Missing,Missing,Missing,int,"{'note': 'str', 'videosCommentedOnSinceAccount...",int,Missing,Missing
2,Activity Summary,"{'ActivitySummaryMap': {'note': 'str', 'videos...",Activity Summary,ActivitySummaryMap,videosSharedSinceAccountRegistration,Missing,Missing,Missing,int,"{'note': 'str', 'videosCommentedOnSinceAccount...",int,Missing,Missing
3,Activity Summary,"{'ActivitySummaryMap': {'note': 'str', 'videos...",Activity Summary,ActivitySummaryMap,videosWatchedToTheEndSinceAccountRegistration,Missing,Missing,Missing,int,"{'note': 'str', 'videosCommentedOnSinceAccount...",int,Missing,Missing
4,Favorite Effects,{'FavoriteEffectsList': 'NoneType'},Favorite Effects,FavoriteEffectsList,NoneType,Missing,Missing,Missing,,NoneType,Missing,Missing,Missing
5,Favorite Hashtags,{'FavoriteHashtagList': 'NoneType'},Favorite Hashtags,FavoriteHashtagList,NoneType,Missing,Missing,Missing,,NoneType,Missing,Missing,Missing
6,Favorite Sounds,{'FavoriteSoundList': 'NoneType'},Favorite Sounds,FavoriteSoundList,NoneType,Missing,Missing,Missing,,NoneType,Missing,Missing,Missing
7,Favorite Videos,"{'App': 'int', 'FavoriteVideoList': [{'Date': ...",Favorite Videos,App,Missing,Missing,Missing,Missing,int,int,Missing,Missing,Missing
8,Favorite Videos,"{'App': 'int', 'FavoriteVideoList': [{'Date': ...",Favorite Videos,FavoriteVideoList,Date,Missing,Missing,Missing,str,"{'Date': 'str', 'Link': 'str'}",str,Missing,Missing
9,Favorite Videos,"{'App': 'int', 'FavoriteVideoList': [{'Date': ...",Favorite Videos,FavoriteVideoList,Link,Missing,Missing,Missing,str,"{'Date': 'str', 'Link': 'str'}",str,Missing,Missing


,variable,value,row_path,col_path_1,col_path_2,col_path_3,col_path_4,col_path_5,data_type,col_path_1_values,col_path_2_values,col_path_3_values,col_path_4_values
0,Activity Summary,"{'ActivitySummaryMap': {'note': 'str', 'videos...",Activity Summary,ActivitySummaryMap,note,Missing,Missing,Missing,str,"{'note': 'str', 'videosCommentedOnSinceAccount...",str,Missing,Missing
1,Activity Summary,"{'ActivitySummaryMap': {'note': 'str', 'videos...",Activity Summary,ActivitySummaryMap,videosCommentedOnSinceAccountRegistration,Missing,Missing,Missing,int,"{'note': 'str', 'videosCommentedOnSinceAccount...",int,Missing,Missing
2,Activity Summary,"{'ActivitySummaryMap': {'note': 'str', 'videos...",Activity Summary,ActivitySummaryMap,videosSharedSinceAccountRegistration,Missing,Missing,Missing,int,"{'note': 'str', 'videosCommentedOnSinceAccount...",int,Missing,Missing
3,Activity Summary,"{'ActivitySummaryMap': {'note': 'str', 'videos...",Activity Summary,ActivitySummaryMap,videosWatchedToTheEndSinceAccountRegistration,Missing,Missing,Missing,int,"{'note': 'str', 'videosCommentedOnSinceAccount...",int,Missing,Missing
4,Favorite Effects,{'FavoriteEffectsList': 'NoneType'},Favorite Effects,FavoriteEffectsList,NoneType,Missing,Missing,Missing,,NoneType,Missing,Missing,Missing
5,Favorite Hashtags,{'FavoriteHashtagList': 'NoneType'},Favorite Hashtags,FavoriteHashtagList,NoneType,Missing,Missing,Missing,,NoneType,Missing,Missing,Missing
6,Favorite Sounds,{'FavoriteSoundList': 'NoneType'},Favorite Sounds,FavoriteSoundList,NoneType,Missing,Missing,Missing,,NoneType,Missing,Missing,Missing
7,Favorite Videos,"{'App': 'int', 'FavoriteVideoList': [{'Date': ...",Favorite Videos,App,Missing,Missing,Missing,Missing,int,int,Missing,Missing,Missing
8,Favorite Videos,"{'App': 'int', 'FavoriteVideoList': [{'Date': ...",Favorite Videos,FavoriteVideoList,Date,Missing,Missing,Missing,str,"{'Date': 'str', 'Link': 'str'}",str,Missing,Missing
9,Favorite Videos,"{'App': 'int', 'FavoriteVideoList': [{'Date': ...",Favorite Videos,FavoriteVideoList,Link,Missing,Missing,Missing,str,"{'Date': 'str', 'Link': 'str'}",str,Missing,Missing


,variable,value,row_path,col_path_1,col_path_2,col_path_3,col_path_4,col_path_5,data_type,col_path_1_values,col_path_2_values,col_path_3_values,col_path_4_values
0,Activity Summary,"{'ActivitySummaryMap': {'note': 'str', 'videos...",Activity Summary,ActivitySummaryMap,note,Missing,Missing,Missing,str,"{'note': 'str', 'videosCommentedOnSinceAccount...",str,Missing,Missing
1,Activity Summary,"{'ActivitySummaryMap': {'note': 'str', 'videos...",Activity Summary,ActivitySummaryMap,videosCommentedOnSinceAccountRegistration,Missing,Missing,Missing,int,"{'note': 'str', 'videosCommentedOnSinceAccount...",int,Missing,Missing
2,Activity Summary,"{'ActivitySummaryMap': {'note': 'str', 'videos...",Activity Summary,ActivitySummaryMap,videosSharedSinceAccountRegistration,Missing,Missing,Missing,int,"{'note': 'str', 'videosCommentedOnSinceAccount...",int,Missing,Missing
3,Activity Summary,"{'ActivitySummaryMap': {'note': 'str', 'videos...",Activity Summary,ActivitySummaryMap,videosWatchedToTheEndSinceAccountRegistration,Missing,Missing,Missing,int,"{'note': 'str', 'videosCommentedOnSinceAccount...",int,Missing,Missing
4,Favorite Effects,{'FavoriteEffectsList': 'NoneType'},Favorite Effects,FavoriteEffectsList,NoneType,Missing,Missing,Missing,,NoneType,Missing,Missing,Missing
5,Favorite Hashtags,{'FavoriteHashtagList': 'NoneType'},Favorite Hashtags,FavoriteHashtagList,NoneType,Missing,Missing,Missing,,NoneType,Missing,Missing,Missing
6,Favorite Sounds,{'FavoriteSoundList': 'NoneType'},Favorite Sounds,FavoriteSoundList,NoneType,Missing,Missing,Missing,,NoneType,Missing,Missing,Missing
7,Favorite Videos,"{'App': 'int', 'FavoriteVideoList': [{'Date': ...",Favorite Videos,App,Missing,Missing,Missing,Missing,int,int,Missing,Missing,Missing
8,Favorite Videos,"{'App': 'int', 'FavoriteVideoList': [{'Date': ...",Favorite Videos,FavoriteVideoList,Date,Missing,Missing,Missing,str,"{'Date': 'str', 'Link': 'str'}",str,Missing,Missing
9,Favorite Videos,"{'App': 'int', 'FavoriteVideoList': [{'Date': ...",Favorite Videos,FavoriteVideoList,Link,Missing,Missing,Missing,str,"{'Date': 'str', 'Link': 'str'}",str,Missing,Missing


,variable,value,row_path,col_path_1,col_path_2,col_path_3,col_path_4,col_path_5,data_type,col_path_1_values,col_path_2_values,col_path_3_values,col_path_4_values
0,Activity Summary,"{'ActivitySummaryMap': {'note': 'str', 'videos...",Activity Summary,ActivitySummaryMap,note,Missing,Missing,Missing,str,"{'note': 'str', 'videosCommentedOnSinceAccount...",str,Missing,Missing
1,Activity Summary,"{'ActivitySummaryMap': {'note': 'str', 'videos...",Activity Summary,ActivitySummaryMap,videosCommentedOnSinceAccountRegistration,Missing,Missing,Missing,int,"{'note': 'str', 'videosCommentedOnSinceAccount...",int,Missing,Missing
2,Activity Summary,"{'ActivitySummaryMap': {'note': 'str', 'videos...",Activity Summary,ActivitySummaryMap,videosSharedSinceAccountRegistration,Missing,Missing,Missing,int,"{'note': 'str', 'videosCommentedOnSinceAccount...",int,Missing,Missing
3,Activity Summary,"{'ActivitySummaryMap': {'note': 'str', 'videos...",Activity Summary,ActivitySummaryMap,videosWatchedToTheEndSinceAccountRegistration,Missing,Missing,Missing,int,"{'note': 'str', 'videosCommentedOnSinceAccount...",int,Missing,Missing
4,Favorite Effects,{'FavoriteEffectsList': 'NoneType'},Favorite Effects,FavoriteEffectsList,NoneType,Missing,Missing,Missing,,NoneType,Missing,Missing,Missing
5,Favorite Hashtags,{'FavoriteHashtagList': 'NoneType'},Favorite Hashtags,FavoriteHashtagList,NoneType,Missing,Missing,Missing,,NoneType,Missing,Missing,Missing
6,Favorite Sounds,{'FavoriteSoundList': 'NoneType'},Favorite Sounds,FavoriteSoundList,NoneType,Missing,Missing,Missing,,NoneType,Missing,Missing,Missing
7,Favorite Videos,"{'App': 'int', 'FavoriteVideoList': [{'Date': ...",Favorite Videos,App,Missing,Missing,Missing,Missing,int,int,Missing,Missing,Missing
8,Favorite Videos,"{'App': 'int', 'FavoriteVideoList': [{'Date': ...",Favorite Videos,FavoriteVideoList,Date,Missing,Missing,Missing,str,"{'Date': 'str', 'Link': 'str'}",str,Missing,Missing
9,Favorite Videos,"{'App': 'int', 'FavoriteVideoList': [{'Date': ...",Favorite Videos,FavoriteVideoList,Link,Missing,Missing,Missing,str,"{'Date': 'str', 'Link': 'str'}",str,Missing,Missing


In [207]:
# Path to the folder containing CSV files
output_path = f"{main_path}TikTok/Output"

# Get a list of all CSV files in the folder
csv_files = list(Path(output_path).glob("*.csv"))

# Load all CSVs into a list of DataFrames
dfs = [pd.read_csv(file) for file in csv_files]


common_columns = ['variable', 'value', 'row_path', 'col_path_1','col_path_2','col_path_3', 'col_path_4', 'col_path_5','data_type', 'col_path_1_values','col_path_2_values','col_path_3_values', 'col_path_4_values']
#, 'col_path_5_values'

for i, df in enumerate(dfs):
    dtype = df['data_type'].dtype
    unique_vals = df['data_type'].unique()
    print(f"DataFrame {i}: 'data_type' dtype = {dtype}, unique values = {unique_vals}")


# Now perform the merge
merged_df = dfs[0]
for df in dfs[1:]:
    merged_df = merged_df.merge(df, on=common_columns, how="outer")

# Replace 'Missing' with NaN
df_final = merged_df.replace('Missing', np.nan)


# Save the final merged DataFrame
df_final.to_csv(f"{main_path}TikTok/Final/Merged_structures_TT.csv", index=False)

DataFrame 0: 'data_type' dtype = object, unique values = ['str' 'int' nan 'bool']
DataFrame 1: 'data_type' dtype = object, unique values = ['str' 'int' nan 'bool']
DataFrame 2: 'data_type' dtype = object, unique values = ['str' 'int' nan 'bool']
DataFrame 3: 'data_type' dtype = object, unique values = ['str' 'int' nan 'bool']


/tmp/ipykernel_1818962/996059225.py:26: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_final = merged_df.replace('Missing', np.nan)
